# ベイズ推定

## ベイズ更新



### 例：コイン投げ

表が出たら1、裏が出たら0として以下のようなデータがとれたとする。

```python
[1,0,1,1,0,1,0,1,1,0]
```

事前分布にベータ分布、尤度をベルヌーイ分布とする



In [9]:
import numpy as np
import pymc as pm
import arviz as az

# 観測データ
y_obs = np.array([1,0,1,1,0,1,0,1,1,0], dtype=int)  # 0/1
N = y_obs.size

with pm.Model() as model:
    p = pm.Beta("p", alpha=1, beta=1)
    y = pm.Bernoulli("y", p=p, observed=y_obs)
    idata = pm.sample(2000, tune=1000, chains=4, progressbar=False, random_seed=42)

az.summary(idata, var_names=["p"])

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]
Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 1 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p,0.579,0.137,0.324,0.83,0.002,0.001,3260.0,4711.0,1.0


## 点推定

### EAP (expected a posteriori, 事後期待値) 

事後分布の期待値を推定値とする方法

$$
\hat{\theta}_{\text {eap }}
=E[\theta \mid \boldsymbol{x}]
=\int \theta f(\theta \mid \boldsymbol{x}) d \theta=\int \theta \frac{f(\boldsymbol{x} \mid \theta) f(\theta)}{f(\boldsymbol{x})} d \theta
$$

In [10]:
eap = idata.posterior.mean(dim=["chain", "draw"])
float(eap["p"])

0.5793009566171891

### MAP (maximum a posteriori, 事後確率最大値）

事後分布の最大値（mode）を推定値とする

$$
\hat{\theta}_{\text {map }}=\max _\theta f(\theta \mid \boldsymbol{x})
$$

In [11]:
with model:
    map_est = pm.find_MAP()
float(map_est["p"])

Output()

0.5999999988336109

### MED (posterior median, 事後中央値）

事後分布の中央値、すなわち、累積分布関数が0.5になる点を推定値とするもの

$$
F\left(\hat{\theta}_{m e d} \mid x\right)=\int^{\hat{\theta}_{\text {med }}} f(\theta \mid \boldsymbol{x}) d \theta=1 / 2
$$

### 事後分散

EAP推定量の散らばりの目安には、事後分布の分散である **事後分散（posterior variance）** と、その平方根をとった **事後標準偏差（posterior standard deviation）** が用いられる。

$$
V[\theta]
=V[\theta \mid \boldsymbol{x}]
=E\left[\left(\theta-\hat{\theta}_{e a p}\right)^2 \mid \boldsymbol{x}\right]
=\int\left(\theta-\hat{\theta}_{e a p}\right)^2 f(\theta \mid \boldsymbol{x}) d \theta
$$


### 予測分布

パラメータの分布ではなく、データの予測値を出すもの。

#### 条件付き予測分布

モデルの分布を$f(x\mid \theta)$ とした場合、 **条件付き予測分布（conditional predictive distribution）** は $f(x^* \mid \hat\theta)$ である。$\hat\theta$はEAP推定量など事後分布からの推定量を用いる。


#### 事後予測分布

**事後予測分布（posterior predictive distribution）**

$$
f(x^* \mid \boldsymbol{x})=\int_{-\infty}^{+\infty} f(x^* \mid \theta) f(\theta \mid \boldsymbol{x}) d \theta
$$

事後分布 $f(\theta \mid \boldsymbol{x})$ におけるモデル分布 $f(x^* \mid \theta)$ の期待値


## 変分ベイズ

[【徹底解説】変分ベイズをはじめからていねいに | Academaid](https://academ-aid.com/ml/vb)